In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install requests_html

In [ ]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import csv
import sys
import urllib
from urllib.error import HTTPError
from requests_html import HTMLSession

In [ ]:
session = HTMLSession()
off = 0
show = '&show=100'
sd_url = 'https://www.sciencedirect.com/search?qs=animal%20data%20star%20classification%20bag'
r = session.get(sd_url)
page_soup = soup(r.text, "html.parser")
result_count = 0
result_count = int(page_soup.find("span", {"class" : "search-body-results-text"}).text.strip().split()[0])
# print(result_count)
individual_urls = []
while(off<result_count):
  offset = "&offset=" + str(off)
  off += 100
  sd_url = 'https://www.sciencedirect.com/search?qs=animal%20data%20star%20classification%20bag' + show + offset
  r = session.get(sd_url)
  page_soup = soup(r.text, "html.parser")
  containers = page_soup.findAll("a", {"class" : "result-list-title-link"})
  url_prefix = "https://www.sciencedirect.com"

  for container in containers:
    container = container["href"];
    paper_url = url_prefix + container
    individual_urls.append(paper_url)
    print(paper_url)

In [ ]:
print(len(individual_urls))

In [ ]:
url = 'https://www.sciencedirect.com/sdfe/arp/cite'
suffix = '?pii=S0264410X18315482&format=text%2Fx-bibtex&withabstract=true'

In [ ]:
import requests

headers = {
    'user-agent': 'My User Agent 1.0',
}
sd_url = 'https://www.sciencedirect.com/sdfe/arp/cite'
params = {
    'pii': None,
    'format': 'text/x-bibtex',
    'withabstract': 'true',
}

bibs_responses = []
for url in individual_urls:
  ind = len(url) - 1 - url[::-1].index('/')
  pii = url[ind+1:]
  print(pii)
  params['pii'] = pii
  # print(params)
  bibs_responses.append(requests.get(sd_url, headers=headers, params=params).text)
  # print(response.text)

In [ ]:
print(len(bibs_responses))

In [ ]:
import pprint
bibs = []
for bib in bibs_responses[:1]:
  # print(bib)
  bibtex = bib.strip("}").strip("\t ") + '\n'
  # print(bibtex)
  ind = bibtex.index("{")
  bib_dict = {}
  bib_dict["type"] = bibtex[:ind]
  bibtex = bibtex[ind+1:]
  ind = bibtex.index("\n")
  bib_dict["id"] = bibtex[:ind-1]
  bibtex = bibtex[ind+1:]
  while(bibtex != "" and bibtex != "\n"): #len(bibtext) > 3):
    # print(bibtex)
    # print("bib", bibtex, len(bibtex))
    ind = bibtex.index("\n")
    attribute = bibtex[:ind].strip("\t,")
    ind_attr = attribute.index("=")
    bib_dict[attribute[:ind_attr].strip()] = attribute[ind_attr+1:].strip()
    bibtex = bibtex[ind + 1:]
  # pprint.p
  # print(bib_dict)
  bibs.append(bib_dict)

In [ ]:
print(bibs)

In [ ]:
output_json = {}
for bib in range(0, len(bibs)):
  print(bibs[bib])
  output_json[bib] = bibs[bib]

In [ ]:
print(output_json)

In [ ]:
import json
filename = "/content/drive/My Drive/bibtex_sd.json"    
with open(filename, 'w') as outfile:
    json.dump(output_json, outfile)